In [1]:
from pandas import DataFrame, read_sql_query, Int32Dtype, read_excel
from datetime import datetime
from IPython.core.interactiveshell import InteractiveShell
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
InteractiveShell.ast_node_interactivity = "all"

In [2]:
AGYW = f"""
SELECT 
    d.case_id,
    dm.id_patient AS id_patient,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        NOW()) AS age,
    IF(TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) >= 10
            AND TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) <= 14,
        '10-14',
        IF(TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) >= 15
                AND TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) <= 19,
            '15-19',
            IF(TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) >= 20
                    AND TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) <= 24,
                '20-24',
                IF(TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) >= 25
                        AND TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) >= 10
            AND TIMESTAMPDIFF(YEAR,
            d.nan_ki_dat_ou_fet,
            NOW()) <= 14,
        '10-14',
        IF(TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) >= 15
                AND TIMESTAMPDIFF(YEAR,
                d.nan_ki_dat_ou_fet,
                NOW()) <= 17,
            '15-17',
            IF(TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) >= 18
                    AND TIMESTAMPDIFF(YEAR,
                    d.nan_ki_dat_ou_fet,
                    NOW()) <= 24,
                '18-24',
                IF(TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) >= 25
                        AND TIMESTAMPDIFF(YEAR,
                        d.nan_ki_dat_ou_fet,
                        NOW()) <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    d.nan_ki_dat_ou_fet AS dob,
    IF(TIMESTAMPDIFF(MONTH,
            d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
            NOW()) >= 0
            AND TIMESTAMPDIFF(MONTH,
            d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
            NOW()) <= 6,
        '0-6 months',
        IF(TIMESTAMPDIFF(MONTH,
                d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                NOW()) >= 7
                AND TIMESTAMPDIFF(MONTH,
                d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                NOW()) <= 12,
            '07-12 months',
            IF(TIMESTAMPDIFF(MONTH,
                    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                    NOW()) >= 13
                    AND TIMESTAMPDIFF(MONTH,
                    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817,
                    NOW()) <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation,
    d.form_link
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
"""

HTS_SCREENING = f"""
SELECT 
    vhs.*,
    vhs.a7_score + vhs.12_score + vhs.12a_score + vhs.12b_score + vhs.14a_score + vhs.15a_score AS total_score
FROM
    caris_db.view_hts_score vhs;
"""

In [3]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

True

In [4]:
sdata= read_sql_query(AGYW,engine,parse_dates=True)
hts_screening = read_sql_query(HTS_SCREENING,engine,parse_dates=True)

In [8]:
hts_screening.columns
sdata.columns

Index(['case_id', 'id_patient', 'patient_code', 'lastname', 'firstname', 'dob',
       'age', 'age_range', 'ovc_age', 'commune', 'interview_date',
       'a7_ak_kils_w_ap_viv_10_14',
       'si_ou_gen_menaj_deja_ou_konn_f_bagay_av_l',
       'a12_ki_laj_ou_te_genyen_l_ou_te_kmanse_f_bagay_pou_premye_fwa',
       'b12_premye_fwa_ou_te_f_bagay_ske_se_te',
       'ske_ou_svi_ak_kapt_deja_l_w_ap_f_bagay',
       'pandan_douz_dnye_mwa_yo_konbyen_fwa_ou_te_svi_ak_kapt', 'a7_score',
       '12_score', '12a_score', '12b_score', '14a_score', '15a_score',
       'total_score'],
      dtype='object')

Index(['case_id', 'id_patient', 'code', 'first_name', 'last_name', 'age',
       'age_range', 'ovc_age', 'dob', 'month_in_program_range',
       'interview_date', 'e__telefn', 'adress', 'already_in_a_group',
       'actual_id_group', 'actual_group_name', 'actual_id_parenting_group',
       'actual_parenting_group_name', 'actual_hub', 'actual_departement',
       'commune', 'commune_section', 'interviewer_firstname',
       'interviewer_lastname', 'interviewer_role', 'interviewer_other_info',
       'interview_location', 'is_your_parent_alive', 'mothers_name',
       'fathers_name', 'who_is_your_law_parent', 'total', 'organisation',
       'form_link'],
      dtype='object')

In [9]:
hts = hts_screening[["case_id","total_score"]]

In [10]:

sdata.id_patient = sdata.id_patient.astype(Int32Dtype())
sdata.age = sdata.age.astype(Int32Dtype())
SDATA = sdata[sdata.total>=14]
SDATA = sdata[sdata.age_range!="25-29"]
SDATA = sdata[sdata.total!="not_valid_age"]

In [16]:
hts_screening.to_excel('./intermed/hts_screening.xlsx',index=False)
SDATA.to_excel('./intermed/SDATA.xlsx',index=False)

## Special

In [27]:
DONNEE = hts.merge(SDATA,on="case_id",how="left")
DONNEE = DONNEE[DONNEE.total_score>=13]
DONNEE

,case_id,total_score,id_patient,code,first_name,last_name,age,age_range,ovc_age,dob,...,interviewer_role,interviewer_other_info,interview_location,is_your_parent_alive,mothers_name,fathers_name,who_is_your_law_parent,total,organisation,form_link
1,0003c06c-9400-46e3-8bf6-297c1f682608,20,162744,PAP/DRMS/043427801,Nadia,Beauprince,22,20-24,18-24,2000-08-12,...,mentor,None,Fatima,2.0,,Menès Beauprince,Magdaline Dorline,20.0,CARIS,None
2,0006ea85-34b8-4e2c-8dde-cd3eb7b96b76,24,169576,GON/DRMS/062455101,daïsha anasheaël,Petit homme,17,15-19,15-17,2005-06-22,...,mentor,None,sou place,0.0,Fabienne noëlsaint,nanis petit homme,fabienne,30.0,CARIS,None
3,000819c9-0838-4425-ac09-11996734cfdd,24,89468,CAP/DRMS/147181402,Guivena,Etienne,19,15-19,18-24,2003-07-13,...,mentor,None,Lory,2.0,Guilène Delusma,Guilène Delusma,Guilène Delusma,35.0,CARIS,None
4,000851a0-9d15-4153-8072-6df912a22539,20,129653,PAP/DRMS/027352801,Michelanda,Romélus,19,15-19,18-24,2003-11-27,...,mentor,None,Grand-Rue,2.0,Erèse Pierre,None,None,25.0,CRS,https://www.commcarehq.org/a/programme-sante-h...
5,00091255-9047-447a-ba45-85c7533af0e8,32,167556,BOM/DRMS/152445406,chaseline,Charles,14,10-14,10-14,2008-01-08,...,mentor,None,ponsonde,0.0,Nanpwen,Chaleus,Chaleus Charles,31.0,CARIS,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76539,fff667b0-1901-47ca-bac1-cf0d016fad56,30,130230,GON/DRMS/084357501,Jolanda,Mesius,23,20-24,18-24,1999-05-09,...,mentor,None,desarmes,0.0,Mesius Caline,Yves Pierre-Louis,Caline Mesius,55.0,CARIS,None
76545,fffaf49f-0cb3-4204-87b4-c2bbe5180ae3,22,78547,PAP/DRMS/038111522,Schammah,Nord,21,20-24,18-24,2001-07-22,...,mentor,None,Bwadtchout,2.0,Marie Sud Julien,,Marie Sud Julien,29.0,CARIS,None
76546,fffbca7a-a5a3-484d-9419-99e0c2499b49,20,52192,PAP/DRMS/010002532,Altagrace,Bellevue,21,20-24,18-24,2001-02-01,...,ankadr_dreams,None,dikini,0.0,Carole Jn Louis,René Bellevue,gran sè,23.0,CARIS,https://www.commcarehq.org/a/caris-test/report...
76548,fffcff4e-8dd0-4408-b43e-52165485722e,28,121698,PAP/DRMS/190288622,yvenette,luma,24,20-24,18-24,1998-11-09,...,mentor,None,None,2.0,None,None,None,30.0,Caris,https://www.commcarehq.org/a/caris-test/report...


In [25]:
DONNEE.sort_values(by=["total_score"],ascending=False,inplace=True)

In [26]:
DONNEE.to_excel("./final/eligible_hts.xlsx",index=False)